# analysis for getting evidence

In [20]:
import time
import pathlib
from os.path import isfile

import math
import torch
import numpy as np

import models
from utils import *
from data import DataLoader


class config(object):
    def __init__(self):
        self.dataset = 'cifar10'
        self.arch = 'resnet'
        self.layers = 14
        self.ckpt = 'ckpt_best.pth'
        self.bn = False
        self.width_mult = 1.0
        self.cuda = True
        self.types = ['max', 'min', 'avg', 'median', 'threshold']
        self.threshold = 0.7
        self.gpuids = [0]

In [17]:
def main():
    global opt, arch_name, all_dist
    opt = config()

    # set model name
    arch_name = set_arch_name(opt)

    print('\n=> creating model \'{}\''.format(arch_name))
    model = models.__dict__[opt.arch](data=opt.dataset, num_layers=opt.layers,
                                      width_mult=opt.width_mult, batch_norm=opt.bn)

    if model is None:
        print('==> unavailable model parameters!! exit...\n')
        exit()

    # checkpoint file
    ckpt_dir = pathlib.Path('checkpoint')
    dir_path = ckpt_dir / arch_name / opt.dataset
    ckpt_file = dir_path / opt.ckpt

    if isfile(ckpt_file):
        print('==> Loading Checkpoint \'{}\''.format(opt.ckpt))
        checkpoint = load_model(model, ckpt_file,
                                main_gpu=None, use_cuda=False)
        print('===> Loaded Checkpoint \'{}\' (epoch {})'.format(
            opt.ckpt, checkpoint['epoch']))
        print(f'\n==> Get and Calculate distribution of absolute PCC')
        all_dist = get_dist_abs_pcc(model)
        print('\n===> done')
        return
    else:
        print('==> no Checkpoint found at \'{}\''.format(
            opt.ckpt))
        return


def get_dist_abs_pcc(model):
    w_kernel = get_kernel(model, opt)
    num_layer = len(w_kernel)

    dist_dict = {}
    for type in opt.types:
        dist_all = []
        for i in tqdm(range(num_layer), ncols=80, unit='layer'):
            ref_layer = torch.Tensor(w_kernel[i])
            if opt.arch in hasDiffLayersArchs:
                ref_layer = ref_layer.view(-1, 9)
            else:
                ref_layer = ref_layer.view(len(w_kernel[i]), -1)

            ref_length = ref_layer.size()[0]

            ref_mean = ref_layer.mean(dim=1, keepdim=True)
            ref_norm = ref_layer - ref_mean
            ref_norm_sq = (ref_norm * ref_norm).sum(dim=1)
            ref_norm_sq_rt = torch.sqrt(ref_norm_sq)

            dist = []
            for j in range(i+1, num_layer):
                cur_weight = torch.Tensor(w_kernel[j])

                # change kernels to dw-kernel
                if opt.arch in hasDiffLayersArchs:
                    cur_weight = cur_weight.view(-1, 9)
                else:
                    cur_weight = cur_weight.view(len(w_kernel[j]), -1)

                cur_length = cur_weight.size()[0]

                cur_mean = cur_weight.mean(dim=1, keepdim=True)
                cur_norm = cur_weight - cur_mean
                cur_norm_sq_rt = torch.sqrt((cur_norm * cur_norm).sum(dim=1))

                cur_dist = []
                for k in range(cur_length):
                    numer = torch.matmul(cur_norm[k], ref_norm.T)
                    denom = ref_norm_sq_rt * cur_norm_sq_rt[k]
                    pcc = numer / denom
                    abs_pcc = torch.abs(pcc)
                    if type == 'max':
                        cur_dist.append(torch.max(abs_pcc).item())
                    elif type == 'min':
                        cur_dist.append(torch.min(abs_pcc).item())
                    elif type == 'avg':
                        cur_dist.append(torch.mean(abs_pcc).item())
                    elif type == 'median':
                        cur_dist.append(torch.median(abs_pcc).item())
                    elif type == 'threshold':
                        num_over_thr = torch.sum(torch.ge(abs_pcc, opt.threshold)).item()
                        ratio_over_thr = num_over_thr / len(abs_pcc)
                        cur_dist.append(ratio_over_thr)
                dist.append(cur_dist)
            dist_all.append(dist)
        dist_dict[type] = dist_all
    print(dist_dict.keys())
    return dist_dict


if __name__ == '__main__':
    start_time = time.time()
    main()
    elapsed_time = time.time() - start_time
    print("====> total time: {:.2f}s".format(elapsed_time))


  0%|                                                 | 0/13 [00:00<?, ?layer/s]
=> creating model 'resnet14'
==> Loading Checkpoint 'ckpt_best.pth'
===> Loaded Checkpoint 'ckpt_best.pth' (epoch 189)

==> Get and Calculate distribution of absolute PCC
100%|████████████████████████████████████████| 13/13 [00:06<00:00,  1.94layer/s]dict_keys(['max', 'min', 'avg', 'median', 'threshold'])

===> done
====> total time: 28.21s



## Draw total histogram

In [18]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


# make directory
dir_plots = pathlib.Path('Histograms') / arch_name / opt.dataset / 'total'
dir_plots.mkdir(parents=True, exist_ok=True)

print('Drawing total histograms...\n')
for i in tqdm(range(len(all_dist['max'])), ncols=80, unit='layer'):
    for j in range(len(all_dist['max'][i])):
        cur_num = i + j + 1
        num_pcc = len(all_dist['max'][i][j])

        plt.style.use('seaborn-deep')
        fig, ax = plt.subplots(figsize=(8,6), dpi=150)

        list_ymax = []
        for type in all_dist.keys():
            if type == 'threshold':
                continue
            cur_dist = all_dist[type][i][j]
            y_vals, x_vals, e_ = ax.hist(cur_dist, label=type, alpha=0.75, bins=min(num_pcc, 256))
            ymax = round((max(y_vals) / num_pcc) + 0.02, 2)
            list_ymax.append(ymax)
        y_max = max(list_ymax)
        ax.set_yticks(ticks=np.arange(0.0, y_max * num_pcc, 0.01 * num_pcc))
        ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
        ax.set_xlim(-0.01, 1.01)
        ax.yaxis.set_major_formatter(PercentFormatter(xmax=num_pcc))
        plt.legend(loc='upper right')
        plt.savefig(dir_plots / 'abs_pcc_ref{:02d}_cur{:02d}.png'.format(i, cur_num),
                    bbox_inches='tight', dpi=150)
        plt.clf()

print('\nDone!!!')


  0%|                                                 | 0/13 [00:00<?, ?layer/s]Drawing total histograms...

100%|████████████████████████████████████████| 13/13 [02:06<00:00,  9.70s/layer]
Done!!!



## Draw all histogram

In [19]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


for type in all_dist.keys():
    # make directory
    type_name = type
    if type == 'threshold':
        type_name += str(opt.threshold)
    dir_plots = pathlib.Path('Histograms') / arch_name / opt.dataset / type_name
    dir_plots.mkdir(parents=True, exist_ok=True)

    print(f'Drawing all histograms (type: {type_name})...\n')
    for i in tqdm(range(len(all_dist[type])), ncols=80, unit='layer'):
        for j in range(len(all_dist[type][i])):
            cur_dist = all_dist[type][i][j]
            num_pcc = len(cur_dist)
            min_pcc = min(cur_dist)
            max_pcc = max(cur_dist)
            med_pcc = np.median(cur_dist)
            avg_pcc = np.mean(cur_dist)
            var_pcc = np.var(cur_dist)
            std_pcc = np.std(cur_dist)
            textstr = '\n'.join((
                r'$\min=%.6f$' % (min_pcc, ),
                r'$\max=%.6f$' % (max_pcc, ),
                r'$\mathrm{median}=%.6f$' % (med_pcc, ),
                r'$\mu=%.6f$' % (avg_pcc, ),
                r'$\sigma^{2}=%.6f$' % (var_pcc, ),
                r'$\sigma=%.6f$' % (std_pcc, )))

            plt.style.use('seaborn-deep')
            fig, ax = plt.subplots(figsize=(8,6), dpi=150)
            cur_num = i + j + 1
            y_vals, x_vals, e_ = ax.hist(cur_dist, bins=min(num_pcc, 256))
            y_max = round((max(y_vals) / num_pcc) + 0.02, 2)
            ax.set_yticks(ticks=np.arange(0.0, y_max * num_pcc, 0.01 * num_pcc))
            ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
            ax.set_xlim(-0.01, 1.01)
            ax.yaxis.set_major_formatter(PercentFormatter(xmax=num_pcc))
            # these are matplotlib.patch.Patch properties
            props = dict(boxstyle='round', facecolor='lightsteelblue', alpha=0.5)
            # place a text box in upper left in axes coords
            ax.text(0.03, 0.96, textstr, transform=ax.transAxes, fontsize=9,
                    verticalalignment='top', bbox=props)
            plt.savefig(dir_plots / 'abs_pcc_ref{:02d}_cur{:02d}.png'.format(i, cur_num),
                        bbox_inches='tight', dpi=150)
            plt.clf()

print('\nDone!!!')

  0%|                                                 | 0/13 [00:00<?, ?layer/s]Drawing all histograms (type: max)...

  0%|                                                 | 0/13 [00:00<?, ?layer/s]Drawing all histograms (type: min)...

  0%|                                                 | 0/13 [00:00<?, ?layer/s]Drawing all histograms (type: avg)...

  0%|                                                 | 0/13 [00:00<?, ?layer/s]Drawing all histograms (type: median)...

  0%|                                                 | 0/13 [00:00<?, ?layer/s]Drawing all histograms (type: threshold0.4)...

100%|████████████████████████████████████████| 13/13 [00:51<00:00,  3.98s/layer]
Done!!!



## Draw total merge histogram

In [19]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


# make directory
dir_plots = pathlib.Path('Histograms') / arch_name / opt.dataset / 'merged'
dir_plots.mkdir(parents=True, exist_ok=True)

print('Drawing total histograms...\n')
plt.style.use('seaborn-deep')
fig, axs = plt.subplots(nrows=opt.layers-1, ncols=opt.layers-1, figsize=(80,60), dpi=150)
for i in tqdm(range(len(all_dist['max'])), ncols=80, unit='layer'):
    for j in range(len(all_dist['max'][i])):
        cur_num = i + j + 1
        num_pcc = len(all_dist['max'][i][j])

        list_ymax = []
        for type in all_dist.keys():
            if type == 'threshold':
                continue
            cur_dist = all_dist[type][i][j]
            y_vals, x_vals, e_ = axs[i,cur_num].hist(cur_dist, label=type,
                alpha=0.75, bins=min(num_pcc, 256))
            ymax = round((max(y_vals) / num_pcc) + 0.02, 2)
            list_ymax.append(ymax)
        y_max = max(list_ymax)
        axs[i,cur_num].set_yticks(ticks=np.arange(0.0, y_max * num_pcc, 0.01 * num_pcc))
        axs[i,cur_num].set_ylim(axs[i,cur_num].get_yticks()[0], axs[i,cur_num].get_yticks()[-1])
        axs[i,cur_num].set_xlim(-0.01, 1.01)
        axs[i,cur_num].yaxis.set_major_formatter(PercentFormatter(xmax=num_pcc))
        if i == 0 and j == len(all_dist['max'][i]) - 1:
            axs[i,cur_num].legend(loc='center', bbox_to_anchor=(1.2, 0), ncol=1, fontsize=15)
        # plt.legend(loc='upper right')
# plt.tight_layout()
plt.savefig(dir_plots / 'total.png', bbox_inches='tight', dpi=150)
# plt.show()
plt.clf()

Drawing total histograms...

100%|████████████████████████████████████████| 13/13 [00:47<00:00,  3.63s/layer]


## Draw histograms of weights each layer

In [23]:
import time
import pathlib
from os.path import isfile

import math
import torch
import numpy as np

import models
from utils import *
from data import DataLoader

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


class config(object):
    def __init__(self):
        self.dataset = 'cifar10'
        self.arch = 'resnet'
        self.layers = 14
        self.ckpt = 'ckpt_best.pth'
        self.bn = False
        self.width_mult = 1.0
        self.cuda = True
        self.types = ['max', 'min', 'avg', 'median', 'threshold']
        self.threshold = 0.4
        self.gpuids = [0]


def main():
    opt = config()

    # set model name
    arch_name = set_arch_name(opt)

    print('\n=> creating model \'{}\''.format(arch_name))
    model = models.__dict__[opt.arch](data=opt.dataset, num_layers=opt.layers,
                                      width_mult=opt.width_mult, batch_norm=opt.bn)

    if model is None:
        print('==> unavailable model parameters!! exit...\n')
        exit()

    # checkpoint file
    ckpt_dir = pathlib.Path('checkpoint')
    dir_path = ckpt_dir / arch_name / opt.dataset
    ckpt_file = dir_path / opt.ckpt

    if isfile(ckpt_file):
        print('==> Loading Checkpoint \'{}\''.format(opt.ckpt))
        checkpoint = load_model(model, ckpt_file,
                                main_gpu=None, use_cuda=False)
        print('===> Loaded Checkpoint \'{}\' (epoch {})'.format(
            opt.ckpt, checkpoint['epoch']))
    else:
        print('==> no Checkpoint found at \'{}\''.format(
            opt.ckpt))
        return

    # make directory
    dir_plots = pathlib.Path('Histograms') / arch_name / opt.dataset / 'conv_weights'
    dir_plots.mkdir(parents=True, exist_ok=True)

    w_kernel = get_kernel(model, opt)
    num_layer = len(w_kernel)

    print('Drawing convolution weights histogram...\n')
    for i in tqdm(range(num_layer), ncols=80, unit='layer'):
        cur_w = np.reshape(w_kernel[i], (-1)).tolist()
        num_w = len(cur_w)
        min_w = min(cur_w)
        max_w = max(cur_w)
        med_w = np.median(cur_w)
        avg_w = np.mean(cur_w)
        var_w = np.var(cur_w)
        std_w = np.std(cur_w)
        textstr = '\n'.join((
            r'$\mathrm{\# weights}=%d$' % (num_w, ),
            r'$\min=%.6f$' % (min_w, ),
            r'$\max=%.6f$' % (max_w, ),
            r'$\mathrm{median}=%.6f$' % (med_w, ),
            r'$\mu=%.6f$' % (avg_w, ),
            r'$\sigma^{2}=%.6f$' % (var_w, ),
            r'$\sigma=%.6f$' % (std_w, )))

        plt.style.use('seaborn-deep')
        fig, ax = plt.subplots(figsize=(8,6), dpi=150)
        y_vals, x_vals, e_ = ax.hist(cur_w, alpha=0.75, bins=min(num_w, 256))
        y_max = round((max(y_vals) / num_w) + 0.02, 2)
        ax.set_yticks(ticks=np.arange(0.0, y_max * num_w, 0.01 * num_w))
        ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
        ax.yaxis.set_major_formatter(PercentFormatter(xmax=num_w))
        # these are matplotlib.patch.Patch properties
        props = dict(boxstyle='round', facecolor='lightsteelblue', alpha=0.5)
        # place a text box in upper left in axes coords
        ax.text(0.03, 0.96, textstr, transform=ax.transAxes, fontsize=9,
                verticalalignment='top', bbox=props)
        plt.savefig(dir_plots / 'Weights_in_Layer{0:02d}.png'.format(i),
                    bbox_inches='tight', dpi=150)
        plt.clf()


if __name__ == '__main__':
    start_time = time.time()
    main()
    elapsed_time = time.time() - start_time
    print("====> total time: {:.2f}s".format(elapsed_time))

  0%|                                                 | 0/13 [00:00<?, ?layer/s]
=> creating model 'resnet14'
==> Loading Checkpoint 'ckpt_best.pth'
===> Loaded Checkpoint 'ckpt_best.pth' (epoch 189)
Drawing convolution weights histogram...

100%|████████████████████████████████████████| 13/13 [00:09<00:00,  1.42layer/s]====> total time: 9.16s



In [2]:
torch.__version__

'1.6.0'

## zip histogram folder

In [10]:
import zipfile
import os
plots_zip = zipfile.ZipFile('Histograms.zip', 'w')
 
for folder, subfolders, files in os.walk('Histograms'):
    for file in files:
        if file.endswith('.png'):
            plots_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder, file), 'Histograms'), compress_type = zipfile.ZIP_DEFLATED)

plots_zip.close()

KeyboardInterrupt: 

## torch profiler test

In [14]:
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.autograd.profiler as profiler

def main():
    global opt, arch_name, all_dist
    opt = config()

    # set model name
    arch_name = set_arch_name(opt)

    print('\n=> creating model \'{}\''.format(arch_name))
    model = models.__dict__[opt.arch](data=opt.dataset, num_layers=opt.layers,
                                      width_mult=opt.width_mult, batch_norm=opt.bn)

    if model is None:
        print('==> unavailable model parameters!! exit...\n')
        exit()

    if opt.cuda:
        torch.cuda.set_device(opt.gpuids[0])
        with torch.cuda.device(opt.gpuids[0]):
            model = model.cuda()
        model = nn.DataParallel(model, device_ids=opt.gpuids,
                                output_device=opt.gpuids[0])
        cudnn.benchmark = True

    # checkpoint file
    ckpt_dir = pathlib.Path('checkpoint')
    dir_path = ckpt_dir / arch_name / opt.dataset
    ckpt_file = dir_path / opt.ckpt

    if isfile(ckpt_file):
        print('==> Loading Checkpoint \'{}\''.format(opt.ckpt))
        checkpoint = load_model(model, ckpt_file,
                                main_gpu=opt.gpuids[0], use_cuda=opt.cuda)
        print('===> Loaded Checkpoint \'{}\' (epoch {})'.format(
            opt.ckpt, checkpoint['epoch']))
        inputs = torch.randn(256, 3, 32, 32).cuda()
        with profiler.profile(use_cuda=True, profile_memory=True, record_shapes=True) as prof:
            model(inputs)
        print(prof.key_averages().table(sort_by="cuda_memory_usage"))
        prof.export_chrome_trace("trace.json")
        return
    else:
        print('==> no Checkpoint found at \'{}\''.format(
            opt.ckpt))
        return


if __name__ == '__main__':
    start_time = time.time()
    main()
    elapsed_time = time.time() - start_time
    print("====> total time: {:.2f}s".format(elapsed_time))



=> creating model 'resnet14'
==> Loading Checkpoint 'ckpt_best.pth'
===> Loaded Checkpoint 'ckpt_best.pth' (epoch 189)
--------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                        Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CUDA total %     CUDA total       CUDA time avg    CPU Mem          Self CPU Mem     CUDA Mem         Self CUDA Mem    Number of Calls  
--------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
empty                       6.73%            660.294us        6.73%            660.294us        5.412us          1.19%           

## heatmap of number of most similar kernel

In [ ]:
import time
import pathlib
from os.path import isfile

import math
import torch
import numpy as np

import models
from utils import *
from data import DataLoader

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


class config(object):
    def __init__(self):
        self.dataset = 'cifar10'
        self.arch = 'resnet'
        self.layers = 14
        self.ckpt = 'ckpt_best.pth'
        self.bn = False
        self.width_mult = 1.0
        self.cuda = True
        self.threshold = 0.4
        self.gpuids = [0]


def main():
    global opt, arch_name, all_dist
    opt = config()

    # set model name
    arch_name = set_arch_name(opt)

    print('\n=> creating model \'{}\''.format(arch_name))
    model = models.__dict__[opt.arch](data=opt.dataset, num_layers=opt.layers,
                                      width_mult=opt.width_mult, batch_norm=opt.bn)

    if model is None:
        print('==> unavailable model parameters!! exit...\n')
        exit()

    # checkpoint file
    ckpt_dir = pathlib.Path('checkpoint')
    dir_path = ckpt_dir / arch_name / opt.dataset
    ckpt_file = dir_path / opt.ckpt

    if isfile(ckpt_file):
        print('==> Loading Checkpoint \'{}\'..'.format(opt.ckpt))
        checkpoint = load_model(model, ckpt_file,
                                main_gpu=None, use_cuda=False)
        print('===> Loaded Checkpoint \'{}\' (epoch {})'.format(
            opt.ckpt, checkpoint['epoch']))
        print(f'\n==> Get and Calculate distribution of absolute PCC..')
        all_dist = get_dist_abs_pcc(model)
        print('\n===> done')
        print('\n==> Draw histogram..')
        histograms(all_dist)
        return
    else:
        print('==> no Checkpoint found at \'{}\''.format(
            opt.ckpt))
        return


def get_dist_abs_pcc(model):
    w_kernel = get_kernel(model, opt)
    num_layer = len(w_kernel)

    dist_all = []
    for i in tqdm(range(num_layer), ncols=80, unit='layer'):
        ref_layer = torch.Tensor(w_kernel[i])
        if opt.arch in hasDiffLayersArchs:
            ref_layer = ref_layer.view(-1, 9)
        else:
            ref_layer = ref_layer.view(len(w_kernel[i]), -1)

        ref_length = ref_layer.size()[0]

        ref_mean = ref_layer.mean(dim=1, keepdim=True)
        ref_norm = ref_layer - ref_mean
        ref_norm_sq_rt = torch.sqrt((ref_norm * ref_norm).sum(dim=1))

        dist = []
        for j in range(i+1, num_layer):
            cur_weight = torch.Tensor(w_kernel[j])

            # change kernels to dw-kernel
            if opt.arch in hasDiffLayersArchs:
                cur_weight = cur_weight.view(-1, 9)
            else:
                cur_weight = cur_weight.view(len(w_kernel[j]), -1)

            cur_length = cur_weight.size()[0]

            cur_mean = cur_weight.mean(dim=1, keepdim=True)
            cur_norm = cur_weight - cur_mean
            cur_norm_sq_rt = torch.sqrt((cur_norm * cur_norm).sum(dim=1))

            cur_dist = []
            for k in range(cur_length):
                numer = torch.matmul(cur_norm[k], ref_norm.T)
                denom = ref_norm_sq_rt * cur_norm_sq_rt[k]
                pcc = numer / denom
                abs_pcc = torch.abs(pcc)
                cur_dist.append(torch.max(abs_pcc).item())
            dist.append(cur_dist)
        dist_all.append(dist)
    return dist_all


def histograms(all_dist):
    # make directory
    dir_plots = pathlib.Path('Histograms') / arch_name / opt.dataset / 'heatmap_N_maxpcc'
    dir_plots.mkdir(parents=True, exist_ok=True)

    # calculate
    histogram_dist = []
    heatmap_dist = []
    for j in range(len(all_dist[0])):
        cur_num = j+1
        max_nums = []
        max_layer_nums = []
        for k in range(len(all_dist[0][j])):
            cur_max = 0.0
            max_ref_layer_num = 0
            for i in range(cur_num):
                if cur_max < all_dist[i][j-i][k]:
                    cur_max = all_dist[i][j-i][k]
                    max_ref_layer_num = i
            max_nums.append(cur_max)
            max_layer_nums.append(max_ref_layer_num)
        histogram_dist.append(max_nums)
        heatmap_dist.append(max_layer_nums)

    # draw heatmap
    print('===> Draw heatmap...')
    plt.clf()
    num_layer = len(all_dist)
    heatmap_cnt = np.zeros((num_layer,num_layer))
    for i in range(1, num_layer):
        for j in range(len(heatmap_dist[i-1])):
            similar_layer_num = heatmap_dist[i-1][j]
            heatmap_cnt[i][similar_layer_num] += 100
        heatmap_cnt[i] = heatmap_cnt[i] / len(heatmap_dist[i-1])
    heatmap_cnt = heatmap_cnt.transpose()
    fig = plt.pcolor(heatmap_cnt, cmap='hot')
    plt.xticks(np.arange(0.5, num_layer, 1), ["{}".format(x) for x in range(num_layer)])
    plt.yticks(np.arange(0.5, num_layer, 1), ["{}".format(x) for x in range(num_layer)])
    plt.xlabel('Source layer', fontsize=12)
    plt.ylabel('Target layer', fontsize=12)
    plt.colorbar()
    plt.savefig(dir_plots / 'heatmap.png', figsize=(8,6), dpi=150, bbox_inches='tight')
    plt.clf()
    print('====> done')

    # draw histograms
    print('===> Draw histograms...')
    for i in tqdm(range(len(histogram_dist)), ncols=80, unit='layer'):
        cur_pcc = histogram_dist[i]
        num_pcc = len(cur_pcc)
        min_pcc = min(cur_pcc)
        max_pcc = max(cur_pcc)
        med_pcc = np.median(cur_pcc)
        avg_pcc = np.mean(cur_pcc)
        var_pcc = np.var(cur_pcc)
        std_pcc = np.std(cur_pcc)
        textstr = '\n'.join((
            r'$\mathrm{\# weights}=%d$' % (num_pcc, ),
            r'$\min=%.6f$' % (min_pcc, ),
            r'$\max=%.6f$' % (max_pcc, ),
            r'$\mathrm{median}=%.6f$' % (med_pcc, ),
            r'$\mu=%.6f$' % (avg_pcc, ),
            r'$\sigma^{2}=%.6f$' % (var_pcc, ),
            r'$\sigma=%.6f$' % (std_pcc, )))

        plt.style.use('seaborn-deep')
        fig, ax = plt.subplots(figsize=(8,6), dpi=150)
        y_vals, x_vals, e_ = ax.hist(cur_pcc, alpha=0.75, bins=min(num_pcc, 256))
        y_max = round((max(y_vals) / num_pcc) + 0.02, 2)
        ax.set_yticks(ticks=np.arange(0.0, y_max * num_pcc, 0.01 * num_pcc))
        ax.set_ylim(ax.get_yticks()[0], ax.get_yticks()[-1])
        ax.set_xlim(-0.01, 1.01)
        ax.yaxis.set_major_formatter(PercentFormatter(xmax=num_pcc))
        # these are matplotlib.patch.Patch properties
        props = dict(boxstyle='round', facecolor='lightsteelblue', alpha=0.5)
        # place a text box in upper left in axes coords
        ax.text(0.03, 0.96, textstr, transform=ax.transAxes, fontsize=9,
                verticalalignment='top', bbox=props)
        plt.savefig(dir_plots / 'Max_PCCs_in_cur{:02d}.png'.format(i+1),
                    bbox_inches='tight', dpi=150)
        plt.clf()
    print('====> done')


if __name__ == '__main__':
    start_time = time.time()
    main()
    elapsed_time = time.time() - start_time
    print("====> total time: {:.2f}s".format(elapsed_time))